# Minimal mnist example on Hopsworks
---

<font color='red'> <h3>Tested with TensorFlow 1.10</h3></font>

## Hops Experiment paradigm <a class="anchor" id='paradigm'></a>

To be able to run your TensorFlow code on Hops, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function. If you wish to run gridsearch over a given set of hyperparameters, you can define arguments for this wrapper function that corresponds to the name of your hyperparameters.

You can also submit one or more `.py`, `.zip` or `.egg` files that contain your code and import them in the wrapper function. To include files, navigate back to HopsWorks and restart restart Jupyter, you can then include files in the Jupyter configuration.

## The `hops` python module

Below you can see the aforementioned wrapper function, which is coincidently named `wrapper` but could potentially be named anything. You can see two imports from the `hops` module, a `tensorboard` and an `hdfs` module. These are the only two modules that you will need to use in your TensorFlow wrapper function. 

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project in HopsFS after each hyperparameter configuration is finished. The `experiment.launch` function, that we will look at abit further down will return the exact path, which you can then navigate to using HopsWorks to inspect the files.

The directory could in practice be used to store other data that should be accessible after each hyperparameter configuration is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```


### Using the `hdfs` module
The `hdfs` module provides a single method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, which is created automatically for each project, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`
```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

![image11-Dataset-ProjectPath.png](../../images/datasets.png)

In [ ]:
def allreduce_mnist():

    import tensorflow as tf
    from hops import devices
    from hops import tensorboard

    PREDICT = tf.estimator.ModeKeys.PREDICT
    EVAL = tf.estimator.ModeKeys.EVAL
    TRAIN = tf.estimator.ModeKeys.TRAIN
    learning_rate=0.002
    batch_size=128
    training_steps=5000

    def build_estimator(config):
        """
        Build the estimator based on the given config and params.
        Args:
            config (RunConfig): RunConfig object that defines how to run the Estimator.
            params (object): hyper-parameters (can be argparse object).
        """
        return tf.estimator.Estimator(
            model_fn=model_fn,
            config=config,
        )


    def model_fn(features, labels, mode):
        """Model function used in the estimator.
        Args:
            features (Tensor): Input features to the model.
            labels (Tensor): Labels tensor for training and evaluation.
            mode (ModeKeys): Specifies if training, evaluation or prediction.
            params (object): hyper-parameters (can be argparse object).
        Returns:
            (EstimatorSpec): Model to be run by Estimator.
        """
        
        features = tf.cast(features, tf.float32)
        # Define model's architecture
        logits = architecture(features, mode)
        class_predictions = tf.argmax(logits, axis=-1)
        # Setup the estimator according to the phase (Train, eval, predict)
        loss = None
        train_op = None
        eval_metric_ops = {}
        predictions = class_predictions
        # Loss will only be tracked during training or evaluation.
        if mode in (TRAIN, EVAL):
            loss = tf.losses.sparse_softmax_cross_entropy(
                labels=tf.cast(labels, tf.int32),
                logits=logits)
        # Training operator only needed during training.
        if mode == TRAIN:
            train_op = get_train_op_fn(loss)
        # Evaluation operator only needed during evaluation
        if mode == EVAL:
            eval_metric_ops = {
                'accuracy': tf.metrics.accuracy(
                    labels=labels,
                    predictions=class_predictions,
                    name='accuracy')
            }
        # Class predictions and probabilities only needed during inference.
        if mode == PREDICT:
            predictions = {
                'classes': class_predictions,
                'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
            }
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=eval_metric_ops
        )


    def architecture(inputs, mode, scope='MnistConvNet'):
        """Return the output operation following the network architecture.
        Args:
            inputs (Tensor): Input Tensor
            mode (ModeKeys): Runtime mode (train, eval, predict)
            scope (str): Name of the scope of the architecture
        Returns:
             Logits output Op for the network.
        """
        with tf.variable_scope(scope):
            inputs = inputs / 255
            input_layer = tf.reshape(inputs, [-1, 28, 28, 1])
            conv1 = tf.layers.conv2d(
                inputs=input_layer,
                filters=20,
                kernel_size=[5, 5],
                padding='valid',
                activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
            conv2 = tf.layers.conv2d(
                inputs=pool1,
                filters=40,
                kernel_size=[5, 5],
                padding='valid',
                activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
            flatten = tf.reshape(pool2, [-1, 4 * 4 * 40])
            dense1 = tf.layers.dense(inputs=flatten, units=256, activation=tf.nn.relu)
            dropout = tf.layers.dropout(
                inputs=dense1, rate=0.5, training=mode==tf.estimator.ModeKeys.TRAIN)
            dense2 = tf.layers.dense(inputs=dropout, units=10)
            return dense2


    def get_train_op_fn(loss):
        """Get the training Op.
        Args:
             loss (Tensor): Scalar Tensor that represents the loss function.
             params (object): Hyper-parameters (needs to have `learning_rate`)
        Returns:
            Training Op
        """
        optimizer = tf.train.AdamOptimizer(learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return train_op


    def get_train_inputs(batch_size, mnist_data):
        """Return the input function to get the training data.
        Args:
            batch_size (int): Batch size of training iterator that is returned
                              by the input function.
            mnist_data ((array, array): Mnist training data as (inputs, labels).
        Returns:
            DataSet: A tensorflow DataSet object to represent the training input
                     pipeline.
        """
        dataset = tf.data.Dataset.from_tensor_slices(mnist_data)
        dataset = dataset.shuffle(
            buffer_size=1000, reshuffle_each_iteration=True
        ).repeat(count=None).batch(batch_size)
        return dataset


    def get_eval_inputs(batch_size, mnist_data):
        """Return the input function to get the validation data.
        Args:
            batch_size (int): Batch size of validation iterator that is returned
                              by the input function.
            mnist_data ((array, array): Mnist test data as (inputs, labels).
        Returns:
            DataSet: A tensorflow DataSet object to represent the validation input
                     pipeline.
        """
        dataset = tf.data.Dataset.from_tensor_slices(mnist_data)
        dataset = dataset.batch(batch_size)
        return dataset
    

     # Read parameters and input data
    mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
        
    config = tf.estimator.RunConfig(
        experimental_distribute=tf.contrib.distribute.DistributeConfig(
            train_distribute=tf.contrib.distribute.CollectiveAllReduceStrategy(
                num_gpus_per_worker=devices.get_num_gpus()),
            eval_distribute=tf.contrib.distribute.MirroredStrategy(
                num_gpus_per_worker=devices.get_num_gpus())),
            model_dir=tensorboard.logdir(),
            save_summary_steps=100,
            log_step_count_steps=100,
            save_checkpoints_steps=500)
        # Setup the Estimator
    model_estimator = build_estimator(config)
    # Setup and start training and validation
    train_spec = tf.estimator.TrainSpec(
         input_fn=lambda: get_train_inputs(batch_size, mnist_train),
         max_steps=training_steps)
    eval_spec = tf.estimator.EvalSpec(
         input_fn=lambda: get_eval_inputs(batch_size, mnist_test),
         steps=None,
         start_delay_secs=10,  # Start evaluating after 10 sec.
         throttle_secs=30  # Evaluate only every 30 sec
    )
        
    tf.estimator.train_and_evaluate(model_estimator, train_spec, eval_spec)


In [ ]:
from hops import experiment
from hops import hdfs

notebook = hdfs.project_path() + "Jupyter/Distributed_Training/collective_allreduce_strategy/mnist.ipynb"
experiment.allreduce(allreduce_mnist,
                  name='mnist estimator', 
                  description='A minimal mnist example with two hidden layers',
                  versioned_resources=[notebook], local_logdir=True)

## Monitoring execution - TensorBoard <a class="anchor" id='tensorboard'></a>
To find the TensorBoard for the execution, please go back to HopsWorks and enter the Experiments service.
Then copy & paste the experiment_id into the textbox and press enter to start a TensorBoard to see all experiments being run in parallel.

![Image7-Monitor.png](../../images/experiments_service.png)
![Image7-Monitor.png](../../images/tensorboard.png)